# AIE425 Intelligent Recommender Systems, Fall Semester 25/26
Assignment 1: Neighborhood CF & Clustering in CF
| Name | ID | Part |
| ------ | -----| -----|
| Youssef Husseiny | 222101943 | Section 3 |
| Habiba Ahmed | 222100471 | Section 2 |
| Seif Amr Abdelhafez | 222102312 | Section 1 |

## Part 2: Item-Based Collaborative Filtering Requirements and Questions:


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
m = pd.read_csv('../../dataset/movies.csv')
r = pd.read_csv('../../dataset/ratings.csv')

In [ ]:
import os

In [ ]:
os.getcwd()

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\section2_neighborhood_cf\\part1_user_based_cf'

In [ ]:
import os
import sys

utils_path = os.path.abspath(os.path.join(os.getcwd(), '../..', 'utils'))

In [ ]:
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [ ]:
print(f"Calculated utils_path: {utils_path}")
print(f"Is utils_path in sys.path: {utils_path in sys.path}")

Calculated utils_path: d:\My_Laptop\University\GU\AI Science\Year 4\Semester 1\Intelligent Recommender System\Assignments\Assignment1\utils
Is utils_path in sys.path: True


In [ ]:
os.listdir(utils_path)

['Cosine_Sim.py',
 'MeanCenteredSimilarityRS.py',
 'Pearson_Sim.py',
 '__init__.py',
 '__pycache__']

In [ ]:
utils_path

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\utils'

In [ ]:
print(f"Original Data: {r['userId'].nunique()} Users, {r['movieId'].nunique()} Movies")

Original Data: 200948 Users, 84432 Movies


In [ ]:
def read_ids_from_file(filepath):
    """Reads IDs from a file."""
    if not os.path.exists(filepath):
        print(f"Warning: File not found {filepath}")
        return []
    with open(filepath, 'r') as f:
        content = f.read().replace('\n', ',') 
        return [int(float(x)) for x in content.split(',') if x.strip().replace('.','',1).isdigit()]

In [ ]:
target_users_ids = read_ids_from_file('../../Results/target_users.txt')
target_items_ids = read_ids_from_file('../../Results/target_items.txt')

In [ ]:
print(f"Loaded Target Users: {target_users_ids}")
print(f"Loaded Target Items: {target_items_ids}")

Loaded Target Users: [79060, 183483, 55243]
Loaded Target Items: [203146, 230591]


In [ ]:
top_movies = r['movieId'].value_counts().head(1200).index.tolist()
target_user_movies = r[r['userId'].isin(target_users_ids)]['movieId'].unique().tolist()
movies_to_keep = list(set(top_movies) | set(target_items_ids) | set(target_user_movies))
r_filtered = r[r['movieId'].isin(movies_to_keep)]

In [ ]:
top_users = r_filtered['userId'].value_counts().head(110000).index.tolist()
users_to_keep = list(set(top_users) | set(target_users_ids))

r_filtered = r_filtered[r_filtered['userId'].isin(users_to_keep)]

In [ ]:
print("Creating Item-User Matrix (Rows=Items)...")
item_matrix_df = r_filtered.pivot_table(index='movieId', columns='userId', values='rating')
item_matrix_np = item_matrix_df.values

item_ids = item_matrix_df.index.tolist()    
user_ids = item_matrix_df.columns.tolist()

In [ ]:
print(f"Users: {r_filtered['userId'].nunique()} (criteria: >= 100,000)")
print(f"Products: {r_filtered['movieId'].nunique()} (criteria: >= 1,000)")
print(f"Ratings: {len(r_filtered)} (criteria: >= 1,000,000)")

Users: 110003 (criteria: >= 100,000)
Products: 1216 (criteria: >= 1,000)
Ratings: 17728444 (criteria: >= 1,000,000)


In [ ]:
item_matrix_np.shape

### Case 1

#### 1. Apply item-based collaborative filtering using Cosine similarity with mean-centering.


In [1]:
from AdjustedCosine_ItemSim import AdjustedCosine_ItemBased_RS

ModuleNotFoundError: No module named 'AdjustedCosine_ItemSim'

In [ ]:
adj_item_rs = AdjustedCosine_ItemBased_RS(item_matrix_np)
adj_item_rs.fit()

In [ ]:
def get_adjusted_cosine_neighbors(item_id, top_n=10):
    """Get Top-N similar items using Adjusted Cosine Similarity."""
        
    i_idx = item_ids.index(item_id)
    
    sims = adj_item_rs.compute_similarities(i_idx)
    
    sorted_idx = np.argsort(sims)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<5} | {'Similar Movie ID':<18} | {'Adj Cosine':<12}")
    print("-" * 50)
    
    count = 0
    for idx in sorted_idx:
        score = sims[idx]
        if score <= 0: continue
            
        similar_id = item_ids[idx]
        print(f"{count+1:<5} | {similar_id:<18} | {score:.6f}")
        
        count += 1
        if count >= 10: break
    
    if count == 0:
        print("  No similar movies found.")

In [ ]:
get_adjusted_cosine_neighbors(target_items_ids[0])

In [ ]:
get_adjusted_cosine_neighbors(target_items_ids[1])  

#### 2. Identify the top 20% of similar items for each target item.


In [ ]:
def get_item_top_20(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return []
    
    i_idx = item_ids.index(item_id)
    
    all_sims = adj_item_rs.compute_similarities(i_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    total_items = len(item_ids)
    top_20_count = int(total_items * 0.20)
    
    top_20_indices = sorted_indices[:top_20_count]
    
    top_20_items = []
    for idx in top_20_indices:
        score = all_sims[idx]
        similar_item_id = item_ids[idx]
        
        if score > 0:
            top_20_items.append((similar_item_id, score))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Total Movies in Matrix: {total_items}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Items with Positive Sim in Top 20%: {len(top_20_items)}")
    
    print(f"\n{'- Preview of Top 10 Similar Movies -':^50}")
    print(f"{'Rank':<5} | {'Movie ID':<15} | {'Adj Cosine':<10}")
    print("-" * 50)
    
    for rank, (sim_id, score) in enumerate(top_20_items[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.5f}")
        
    print("-" * 50)
    return top_20_items

In [ ]:
top_20_items_1 = get_item_top_20(target_items_ids[0])

In [ ]:
top_20_items_2 = get_item_top_20(target_items_ids[1])

#### 3. Predict missing ratings using these items.

In [ ]:
def generate_item_recommendations(uid,K = 10):
    if uid not in user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = user_ids.index(uid)
    user_ratings = item_matrix_np[:, u_idx]
    unrated_item_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_item_indices)}")
    print("Calculating Item-Based predictions... (this might take a minute)")
    
    recommendations = []
    
    for i_idx in unrated_item_indices:
        try:
            pred_val = adj_item_rs.pred(u_idx=u_idx, i_idx=i_idx, k=K)
            
            if pred_val > 0:
                item_id = item_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Item-Based):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Predicted Rating':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations

In [ ]:
item_recs_u1 = generate_item_recommendations(target_users_ids[0])

In [ ]:
item_recs_u2 = generate_item_recommendations(target_users_ids[1])

#### 4. Compute DF and DS.


In [ ]:
from AdjustedCosine_ItemSim import Discounted_AdjustedCosine_ItemBased_RS


In [ ]:
d_adj_item_rs = Discounted_AdjustedCosine_ItemBased_RS(item_matrix_np)
d_adj_item_rs.fit()

In [ ]:
total_users = item_matrix_np.shape[1] 
beta_percentage = 0.30
beta_threshold = int(np.ceil(total_users * beta_percentage))

In [ ]:
def get_discounted_similar_items(item_id):
    """Get Top-N similar items using Discounted Adjusted Cosine Similarity."""
        
    i_idx = item_ids.index(item_id)
    
    raw_sims, counts, dfs, dses = d_adj_item_rs.get_discounted_stats(i_idx, beta_threshold)
    
    sorted_indices = np.argsort(raw_sims)[::-1]
    
    print(f"\nTarget Movie: {item_id}")
    print(f"{'Rank':<4} | {'Similar Item':<18} | {'Common Users':<12} | {'DF':<6} | {'Adj Cos':<10} | {'-->':<3} | {'DS (Disc)':<10}")
    print("-" * 90)
    
    count = 0
    for idx in sorted_indices:
        if raw_sims[idx] <= 0: continue
        
        sim_id = item_ids[idx]
        n_raw = raw_sims[idx]
        n_count = int(counts[idx])
        n_df = dfs[idx]
        n_ds = dses[idx]
        
        print(f"{count+1:<4} | {sim_id:<18} | {n_count:<12} | {n_df:.4f} | {n_raw:.6f} | --> | {n_ds:.6f}")
        
        count += 1
        if count >= 10: break
        
    if count == 0:
        print("  No similar items found.")

In [ ]:
get_discounted_similar_items(target_users_ids[0])  

In [ ]:
get_discounted_similar_items(target_users_ids[1])  

#### 5. Select top 20% items using DS.


In [ ]:
total_users = item_matrix_np.shape[1] 
beta_threshold = int(np.ceil(total_users * 0.30))

In [ ]:
def get_item_top_20_discounted(item_id):
    if item_id not in item_ids:
        print(f"Target Item {item_id} missing.")
        return []
    
    i_idx = item_ids.index(item_id)
    _, _, _, dses = d_adj_item_rs.get_discounted_stats(i_idx, beta_threshold)
    
    sorted_indices_ds = np.argsort(dses)[::-1]
    
    total_items = len(item_ids)
    top_20_count = int(total_items * 0.20)
    
    top_20_indices_ds = sorted_indices_ds[:top_20_count]
    
    top_20_items_ds = []
    for idx in top_20_indices_ds:
        sim_id = item_ids[idx]
        score_ds = dses[idx]
        
        if score_ds > 0:
            top_20_items_ds.append((sim_id, score_ds))
            
    print(f"\nTarget Movie: {item_id}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Items with Positive DS in Top 20%: {len(top_20_items_ds)}")
    
    print(f"\n{'- Preview of Top 10 (Discounted) -':^50}")
    print(f"{'Rank':<5} | {'Movie ID':<15} | {'Discounted Sim':<15}")
    print("-" * 50)
    
    for rank, (sim_id, score) in enumerate(top_20_items_ds[:10]):
        print(f"{rank+1:<5} | {sim_id:<15} | {score:.6f}")
        
    print("-" * 50)
    return top_20_items_ds

In [ ]:
item_top_20_ds_1 = get_item_top_20_discounted(target_items_ids[0])

In [ ]:
item_top_20_ds_2 = get_item_top_20_discounted(target_items_ids[1])

#### 6. Use these for updated rating predictions.


In [ ]:
def generate_item_discounted_recommendations(uid,K = 10):
    if uid not in user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = user_ids.index(uid)
    
    user_ratings = d_adj_item_rs.matrix[:, u_idx]
    unrated_item_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_item_indices)}")
    print("Calculating Discounted Item-Based predictions...")
    
    recommendations = []
    
    for i_idx in unrated_item_indices:
        try:
            pred_val = d_adj_item_rs.pred_discounted(u_idx, i_idx, K, beta_threshold)
            if pred_val > 0:
                item_id = item_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Discounted Item-Based):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Disc Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations

In [ ]:
item_disc_recs_u1 = generate_item_discounted_recommendations(target_users_ids[0])

In [ ]:
item_disc_recs_u2 = generate_item_discounted_recommendations(target_users_ids[1])

#### 7. Compare similarity lists from steps 2 and 5. Provide commentary.

In [ ]:
top_20_items_1

In [ ]:
top_20_items_1_df = pd.DataFrame(top_20_items_1, columns=['movieId', 'discounted_similarity'])
top_20_items_1_df

In [ ]:
item_top_20_ds_1_df = pd.DataFrame(item_top_20_ds_1, columns=['movieId', 'discounted_similarity'])
item_top_20_ds_1_df

In [ ]:
top_20_items_1_compare = pd.merge(top_20_items_1_df, item_top_20_ds_1_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_1_compare

In [ ]:
top_20_items_2_df = pd.DataFrame(top_20_items_2, columns=['movieId', 'discounted_similarity'])
top_20_items_2_df

In [ ]:
item_top_20_ds_2_df = pd.DataFrame(item_top_20_ds_2, columns=['movieId', 'discounted_similarity'])
item_top_20_ds_2_df

In [ ]:
top_20_items_2_compare = pd.merge(top_20_items_2_df, item_top_20_ds_2_df, on='movieId', suffixes=('_adj_cosine', '_discounted'))
top_20_items_2_compare

##### Comment


The Discounted list is far more robust. It prioritizes items where the similarity is statistically significant, ensuring that the recommendation is based on a consensus of many users rather than the coincidence of a few.

#### 8. Compare predicted ratings from steps 3 and 6. Discuss.

In [ ]:
item_recs_u1_df = pd.DataFrame(item_recs_u1, columns=['movieId', 'predicted_rating'])
item_recs_u1_df

In [ ]:
item_disc_recs_u1_df = pd.DataFrame(item_disc_recs_u1, columns=['movieId', 'predicted_rating'])
item_disc_recs_u1_df

In [ ]:
item_recs_u2_df = pd.DataFrame(item_recs_u2, columns=['movieId', 'predicted_rating'])
item_recs_u2_df

In [ ]:
item_disc_recs_u2_df = pd.DataFrame(item_disc_recs_u2, columns=['movieId', 'predicted_rating'])
item_disc_recs_u2_df

##### Discussion

The predicted ratings generated using the Discounted Similarity (Task 6) are generally lower or more conservative compared to the predictions from the Raw Similarity (Task 3). because of Filtering Outliers and discount factor weighting

#### 9. Give your comments in a separate section in your report.

- The few common users Problem: Adjusted Cosine Similarity calculates the angle between two vectors. If the vectors are very short (e.g., only 2 common users), it is statistically easy for them to align perfectly by chance.Example: If User A rates both Movie X and Movie Y as 5.0, and User B rates both as 1.0, the math sees a perfect pattern match (Sim = 1.0), even though we only have 2 data points.
- Lack of Variance: Without enough common users, the algorithm cannot distinguish between true preference correlation and random coincidence.
- Role of Discounting: This confirms why computing DF and DS (Task 4) is critical. By setting a threshold (e.g., $\beta = 30\%$), we force the algorithm to treat low-support correlations as "weak," regardless of how high their raw cosine score is. This prevents the system from recommending obscure items based on insufficient evidence.

### Case 2